In [1]:
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle

from tensorflow.keras.callbacks import ModelCheckpoint

# os.environ["CUDA_DEVICE_ORDER"] = "GPU"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from tensorflow.compat.v1.keras import backend as K

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

import wandb
from wandb.keras import WandbCallback

mpl.rcParams['figure.figsize'] = (20, 10)
mpl.rcParams['axes.grid'] = False

cfg = tf.compat.v1.ConfigProto()
cfg.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=cfg)
K.set_session(sess)    

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5502407636528575035
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 890245541389249908
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4925095936
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12976571041280552539
physical_device_desc: "device: 0, name: Quadro P2000, pci bus id: 0000:02:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6166074350547664903
physical_device_desc: "device: XLA_GPU device"
]


In [2]:
df = pickle.load( open( "/home/kriza/programing/clanok/data/picklnute/30min.p", "rb" ) )

In [3]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i)
    # Reshape data from (history_size,) to (history_size, 1)
    data.append(np.reshape(dataset[indices], (history_size, 1)))
    labels.append(dataset[i+target_size])
  return np.array(data), np.array(labels)

In [4]:
TRAIN_SPLIT_BEGIN = 24
TRAIN_SPLIT_END = 15551 - TRAIN_SPLIT_BEGIN
VALID_SPLIT_BEGIN = 15601 - TRAIN_SPLIT_BEGIN
VALID_SPLIT_END = 17463 -TRAIN_SPLIT_BEGIN
tf.random.set_seed(13)

In [5]:
uni_data = df['30m-item61']

In [6]:
uni_data = uni_data.values
uni_data = uni_data[TRAIN_SPLIT_BEGIN:VALID_SPLIT_END + TRAIN_SPLIT_BEGIN]

In [7]:
uni_train_mean = uni_data[:TRAIN_SPLIT_END].mean()
uni_train_std = uni_data[:TRAIN_SPLIT_END].std()
uni_data = (uni_data-uni_train_mean)/uni_train_std

In [8]:
UNIT_COUNT = range(10, 30, 10)
HISTORY = range(20, 30, 10)
BATCH_SIZES = [64, 128, 256, 512]


univariate_past_history = 0
BATCH_SIZE = 0
UNITS = 0
saved_models = ""

for i in BATCH_SIZES:
    for j in HISTORY:
        for k in UNIT_COUNT:
            univariate_past_history = j
            BATCH_SIZE = i
            UNITS = k
            saved_models = "saved_models/" + "hist=" + str(univariate_past_history) + "batch=" + str(BATCH_SIZE) + "units=" + str(UNITS) + "/model"
            wandb.init(project="predikcie1krok", name = "hist=" + str(univariate_past_history) + "batch=" + str(BATCH_SIZE) + "units=" + str(UNITS))
            break
        break
    break



Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable


In [ ]:
# tu cyklus

In [28]:

univariate_future_target = 0

x_train_uni, y_train_uni = univariate_data(uni_data, 0, TRAIN_SPLIT_END,
                                           univariate_past_history,
                                           univariate_future_target)
x_val_uni, y_val_uni = univariate_data(uni_data, VALID_SPLIT_BEGIN, None,
                                       univariate_past_history,
                                       univariate_future_target)

In [29]:

BUFFER_SIZE = 10000

train_univariate = tf.data.Dataset.from_tensor_slices((x_train_uni, y_train_uni))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

val_univariate = tf.data.Dataset.from_tensor_slices((x_val_uni, y_val_uni))
val_univariate = val_univariate.batch(BATCH_SIZE)

In [30]:
simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(UNITS, input_shape=x_train_uni.shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model.compile(optimizer='adam', loss='mae', metrics='mae')

In [31]:
EPOCHS = 10

ckpt = ModelCheckpoint(filepath=saved_models, monitor='val_loss', mode='min', save_best_only = True, save_weights_only=True, verbose = False)

simple_lstm_model.fit(train_univariate, epochs=EPOCHS,
                    #   steps_per_epoch=int(x_train_uni.shape[0]/BATCH_SIZE),
                      validation_data=val_univariate,
                      callbacks=[ckpt, WandbCallback()]
                    #   , validation_steps=50
                      )

Epoch 1/10
Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
243/243 [==============================] - 1s 5ms/step - loss: 0.5914 - mae: 0.5914 - val_loss: 2.7060 - val_mae: 2.7060
Epoch 2/10
243/243 [==============================] - 1s 2ms/step - loss: 0.2360 - mae: 0.2360 - val_loss: 2.0557 - val_mae: 2.0557
Epoch 3/10
243/243 [==============================] - 1s 2ms/step - loss: 0.1915 - mae: 0.1915 - val_loss: 1.8875 - val_mae: 1.8875
Epoch 4/10
243/243 [==============================] - 1s 2ms/step - loss: 0.1808 - mae: 0.1808 - val_loss: 1.7871 - val_mae: 1.7871
Epoch 5/10
243/243 [==============================] - 1s 2ms/step - loss: 0.1772 - mae: 0.1772 - val_loss: 1.7184 - val_mae: 1.7184
Epoch 6/10
243/243 [==============================] - 1s 2ms/step - loss: 0.1753 - mae: 0.1753 - val_loss: 1.6653 - val_mae: 1.6653
Epoch 7/10
243/243 [==============================] - 1s 2ms/step - loss: 0.1742 - mae: 0.1742 - va